# Populating a Relational Database
## DSA Interview Questions
In this example we will build and populate a SQLite database with DSA interview questions.

## Creating the initial database
Let's create our initial tables. First, we will populate the smaller question type and question level tables. Then, we will populate the larger *question* table.

In [1]:
import pandas as pd
import pyarrow as pa
import sqlite3
import os

conn = sqlite3.connect("dsa.db")

In [2]:
# Create a question type table
conn.execute('''
CREATE TABLE IF NOT EXISTS question_type (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL
)
'''
)

# Populate it with the following: Binary Search, Graph, Two Pointers, Dynamic Programming
conn.execute('''
INSERT INTO question_type (name) VALUES
    ('Binary Search'),
    ('Graph'),
    ('Two Pointers'),
    ('Dynamic Programming')
'''
)

# Create a question level table
conn.execute('''
CREATE TABLE IF NOT EXISTS question_level (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL
)
'''
)

# Populate it with the following: Easy, Medium, Hard
conn.execute('''
INSERT INTO question_level (name) VALUES
    ('Easy'),
    ('Medium'),
    ('Hard')
'''
)

# Create a questions table
conn.execute('''
CREATE TABLE IF NOT EXISTS questions (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    link TEXT NOT NULL,
    question_type_id INTEGER,
    question_level_id INTEGER,
    FOREIGN KEY (question_type_id) REFERENCES question_type(id),
    FOREIGN KEY (question_level_id) REFERENCES question_level(id)
)
'''
)

conn.commit()

# Prepare the input to SwellDB
Since we need to populate the questions table, we need to create all the possible combinations of question types and levels.

In [3]:
# Get all combinations
combos = """
SELECT 
  qt.id as question_type_id,
  qt.name as question_type_name,
  ql.id as question_level_id,
  ql.name as question_level_name
FROM question_type as qt
CROSS JOIN question_level as ql
"""

In [4]:
combos = pd.read_sql(combos, conn)
combos

,question_type_id,question_type_name,question_level_id,question_level_name
0,1,Binary Search,1,Easy
1,1,Binary Search,2,Medium
2,1,Binary Search,3,Hard
3,1,Binary Search,4,Easy
4,1,Binary Search,5,Medium
...,...,...,...,...
103,12,Dynamic Programming,5,Medium
104,12,Dynamic Programming,6,Hard
105,12,Dynamic Programming,7,Easy
106,12,Dynamic Programming,8,Medium


In [5]:
# Convert the combinations to a pyarrow table
data = pa.Table.from_pandas(combos)

# SwellDB
Let's populate the *question* table with SwellDB.

In [6]:
import os
import logging

import datafusion

# SwellDB imports
from swelldb import SwellDB, OpenAILLM
from swelldb.swelldb import Mode
from swelldb.table_plan.table.physical.dataset_table import DatasetTable
from swelldb.table_plan.table.physical.llm_table import LLMTable
from swelldb.table_plan.table.physical.search_engine_table import SearchEngineTable

# Initialize a SwellDB instance
swelldb: SwellDB = SwellDB(
    llm=OpenAILLM(api_key=os.environ["OPENAI_API_KEY"], model="gpt-4o"), 
    serper_api_key=os.environ["SERPER_API_KEY"])

In [8]:
questions = (
    swelldb.table_builder()
    .set_table_name("question")
    .set_content("A table that contains DSA questions from Leetcode. Given the input data, create as many as possible.")
    .set_schema("name str, link str, question_type_name str, question_type_id int, question_level_name str, question_level_id int")
    .set_base_columns(["question_type_name", "question_level_name"])
    .set_table_gen_mode(Mode.OPERATORS)
    .set_operators([LLMTable, SearchEngineTable])
    .set_data(data)
    .set_chunk_size(20)
).build()

KeyError: "'question_level_name'"

In [ ]:
logging.basicConfig(level=logging.ERROR)

In [17]:
questions.explain()

SearchEngineTable[schema=['question_level_id', 'question_type_id', 'link', 'question_type_name', 'question_level_name']
--LLMTable[schema=['name', 'question_level_name', 'question_type_name']


In [10]:
# Materialize the table
ds = questions.materialize()

sc = datafusion.SessionContext()
sc.deregister_table("questions")
sc.register_dataset("questions", pa.dataset.dataset(ds))

In [11]:
ds.to_pandas()

,question_level_id,question_type_id,link,question_type_name,question_level_name,name
0,1,1,https://leetcode.com/problem-list/array/,Array,Easy,Two Sum
1,2,2,https://leetcode.com/problem-list/linked-list/,Linked List,Medium,Add Two Numbers
2,2,3,https://leetcode.com/problem-list/string/,String,Medium,Longest Substring Without Repeating Characters
3,3,1,https://leetcode.com/problem-list/array/,Array,Hard,Median of Two Sorted Arrays
4,1,4,https://leetcode.com/problem-list/stack/,Stack,Easy,Valid Parentheses
5,1,2,https://leetcode.com/problem-list/linked-list/,Linked List,Easy,Merge Two Sorted Lists
6,2,5,https://leetcode.com/problem-list/binary-search/,Binary Search,Medium,Search in Rotated Sorted Array
7,3,6,https://leetcode.com/problem-list/breadth-firs...,Breadth-First Search,Hard,Word Ladder
8,2,7,https://leetcode.com/problem-list/depth-first-...,Depth-First Search,Medium,Clone Graph
9,3,8,https://leetcode.com/problem-list/two-pointers/,Two Pointers,Hard,Trapping Rain Water


In [12]:
sc.sql(""" 
SELECT *
FROM questions
""")

question_level_id,question_type_id,link,question_type_name,question_level_name,name
1,1,https://leetcode.com/prob https://leetcode.com/problem-list/array/ ...,Array,Easy,Two Sum
2,2,https://leetcode.com/prob https://leetcode.com/problem-list/linked-list/ ...,Linked List,Medium,Add Two Numbers
2,3,https://leetcode.com/prob https://leetcode.com/problem-list/string/ ...,String,Medium,Longest Substring Without Longest Substring Without Repeating Characters ...
3,1,https://leetcode.com/prob https://leetcode.com/problem-list/array/ ...,Array,Hard,Median of Two Sorted Arra Median of Two Sorted Arrays ...
1,4,https://leetcode.com/prob https://leetcode.com/problem-list/stack/ ...,Stack,Easy,Valid Parentheses
1,2,https://leetcode.com/prob https://leetcode.com/problem-list/linked-list/ ...,Linked List,Easy,Merge Two Sorted Lists
2,5,https://leetcode.com/prob https://leetcode.com/problem-list/binary-search/ ...,Binary Search,Medium,Search in Rotated Sorted Search in Rotated Sorted Array ...
3,6,https://leetcode.com/prob https://leetcode.com/problem-list/breadth-first-search/ ...,Breadth-First Search,Hard,Word Ladder
2,7,https://leetcode.com/prob https://leetcode.com/problem-list/depth-first-search/ ...,Depth-First Search,Medium,Clone Graph
3,8,https://leetcode.com/prob https://leetcode.com/problem-list/two-pointers/ ...,Two Pointers,Hard,Trapping Rain Water


In [13]:
ds.to_pandas().drop(columns=["question_type_name", "question_level_name"]).to_sql("questions", conn, if_exists="append", index=False)

10

In [14]:
query = """
SELECT q.name, ql.name as level, q.link
FROM questions q, question_type qt, question_level ql
WHERE q.question_type_id = qt.id
AND q.question_level_id = ql.id
"""

pd.set_option('display.max_colwidth', 400)

df = pd.read_sql(query, conn)

In [15]:
df

,name,level,link
0,Two Sum,Easy,https://leetcode.com/problem-list/array/
1,Add Two Numbers,Medium,https://leetcode.com/problem-list/linked-list/
2,Longest Substring Without Repeating Characters,Medium,https://leetcode.com/problem-list/string/
3,Median of Two Sorted Arrays,Hard,https://leetcode.com/problem-list/array/
4,Valid Parentheses,Easy,https://leetcode.com/problem-list/stack/
5,Merge Two Sorted Lists,Easy,https://leetcode.com/problem-list/linked-list/
6,Search in Rotated Sorted Array,Medium,https://leetcode.com/problem-list/binary-search/
7,Word Ladder,Hard,https://leetcode.com/problem-list/breadth-first-search/
8,Clone Graph,Medium,https://leetcode.com/problem-list/depth-first-search/
9,Trapping Rain Water,Hard,https://leetcode.com/problem-list/two-pointers/
